# core

> Defines central classes of findmycells: `ProcessingStrategy`, `ProcessingObject`, `DataLoader`, and `DataReader`

In [ ]:
#| default_exp core

In [ ]:
#| export

from abc import ABC, abstractmethod
from findmycells.database import Database
from typing import List, Dict, Tuple, Optional, Any
from types import ModuleType
from pathlib import Path

In [ ]:
#| hide
from nbdev.showdoc import *

The following two classes, `ProcessingObject` and `ProcessingStrategy`, provide the blueprints for all processing strategies and objects that are used throughout the *findmycells* package. As you can see in the corresponding processing step modules (i.e. "preprocess", "segment", or "quantify"), these abstract base classes provide the basic structure of the more specific objects and strategies in each of these modules (e.g. `QuantificationObject` and `QuantificationStrategy` within the "quantify" module inherit from `ProcessingObject` and `ProcessingStrategy`, respectively). While this makes these two classes highly relevant for any developer, regular users of *findmycells* won´t be interacting with them, even if they want to use the API instead of the GUI.

In [ ]:
#| export

class ProcessingObject(ABC):
    
    """
    Abstract base class (inherits from ABC) that defines the general structure of `ProcessingObjects` in findmycells.
    A `ProcessingObject` combines all information needed for the corresponding processing step, 
    i.e. what files are supposed to be processed & how. It also interfaces to the database of the
    project, such that it can automatically update the database with the latest progress.
    """
    
    def __init__(self, 
                 database: Database, # The database of the findmycells project
                 file_ids: List, # A list with the file_ids of all files that need to be processed
                 strategies: List # A list of all ProcessingStrategies that shall be run on the files defined in file_ids.
                ) -> None:
        self.database = database
        self.file_ids = file_ids
        self.strategies = strategies


    @property
    @abstractmethod
    def processing_type(self
                       ) -> str: # string that defines the processing type (e.g. "preprocessing" or "quantification")
        """
        Abstract method that requires its subclasses to define the `processing_type`
        as a property of the class. Thus, this will be specified in each individual 
        processing module (e.g. the "preprocess" or "quantify" modules). It will be used
        in the database to keep track of the processing progress of the project.
        Has to be a string.
        """
        pass


    @abstractmethod
    def add_processing_specific_infos_to_updates(self, 
                                                 updates: Dict # A dictionary with updates that need to be passed to the database
                                                ) -> Dict: # A dictionary with all updates that need to be passed to the database
        """
        Abstract method that that requires its subclasses to define what updates need to be
        passed to the database, in addition to those that are already covered by the corresponding
        ProcessingStrategies or the "self.update_database()" method. If there are no more 
        information to add, simply return the input 'updates' dictionary without any alterations.
        
        Returns a dictionary with all updates that need to be passed to the database.
        """
        return updates
    
    
    def run_all_strategies(self) -> None:
        """
        Runs all ProcessingStrategies that were passed upon initialization (i.e. self.strategies).
        For this, the corresponding ProcessingStrategy objects will be initialized and their ".run()"
        method will be called, while passing "self" as "processing_object". Finally, it updates the
        database and deletes the ProcessingStrategy object to clear it from memory.
        """
        for strategy in self.strategies:
            processing_strategy = strategy()
            self = processing_strategy.run(processing_object = self)
            self = processing_strategy.update_database(processing_object = self)
            del processing_strategy


    def update_database(self) -> None:
        """
        For each microscopy file that had to be processed (self.file_ids), the database
        will be updated with the respective processing progress information. Interfaces
        back to the abstract method "self.add_processing_specific_infos_to_updates()" that
        enables the corresponding subclasses to add more specific details before triggering
        the update method of the database.
        """
        for file_id in self.file_ids:
            updates = dict()
            updates[f'{self.processing_type}_completed'] = True
            updates = self.add_processing_specific_infos_to_updates(updates = updates)
            self.database.update_file_infos(file_id = file_id, updates = updates)

Subclasses that inherit from `ProcessingObject` need to implement the following two abstract methods:

In [ ]:
show_doc(ProcessingObject.processing_type)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L35){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.processing_type

>      ProcessingObject.processing_type ()

Abstract method that requires its subclasses to define the `processing_type`
as a property of the class. Thus, this will be specified in each individual 
processing module (e.g. the "preprocess" or "quantify" modules). It will be used
in the database to keep track of the processing progress of the project.
Has to be a string.

In [ ]:
show_doc(ProcessingObject.add_processing_specific_infos_to_updates)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L48){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.add_processing_specific_infos_to_updates

>      ProcessingObject.add_processing_specific_infos_to_updates (updates:Dict)

Abstract method that that requires its subclasses to define what updates need to be
passed to the database, in addition to those that are already covered by the corresponding
ProcessingStrategies or the "self.update_database()" method. If there are no more 
information to add, simply return the input 'updates' dictionary without any alterations.

Returns a dictionary with all updates that need to be passed to the database.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| updates | typing.Dict | A dictionary with updates that need to be passed to the database |
| **Returns** | **typing.Dict** | **A dictionary with all updates that need to be passed to the database** |

In addition, `ProcessingObject` defines two core functions that will be called on all its subclasses, which are:

In [ ]:
show_doc(ProcessingObject.run_all_strategies)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L62){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.run_all_strategies

>      ProcessingObject.run_all_strategies ()

Runs all ProcessingStrategies that were passed upon initialization (i.e. self.strategies).
For this, the corresponding ProcessingStrategy objects will be initialized and their ".run()"
method will be called, while passing "self" as "processing_object". Finally, it updates the
database and deletes the ProcessingStrategy object to clear it from memory.

In [ ]:
show_doc(ProcessingObject.update_database)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L76){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.update_database

>      ProcessingObject.update_database ()

For each microscopy file that had to be processed (self.file_ids), the database
will be updated with the respective processing progress information. Interfaces
back to the abstract method "self.add_processing_specific_infos_to_updates()" that
enables the corresponding subclasses to add more specific details before triggering
the update method of the database.

In [ ]:
#| export

class ProcessingStrategy(ABC):
    
    """
    Abstract base class that defines the general structure of `ProcessingStrategies` in findmycells.
    A `ProcessingStrategy` combines all functions that are required for one particular processing step, 
    e.g. `ConvertTo8Bit` is a `ProcessingStrategy` in the "preprocess" module and converts the corresponding
    images into 8-bit.
    """

    @property
    @abstractmethod
    def processing_type(self):
        # has to be any of these: 'preprocessing', 'segmentation', 'quantification', 'inspection'
        pass


    @abstractmethod
    def run(self, processing_object: ProcessingObject) -> ProcessingObject:
        # process the processing_object
        return processing_object

    
    @abstractmethod
    def add_strategy_specific_infos_to_updates(self, updates: Dict) -> Dict:
        # add all ProcessingStrategy specifc information to the update dictionary
        # or simply return updates right away if there are no information to add
        return updates
    def update_database(self, processing_object: ProcessingObject) -> ProcessingObject:
        for file_id in processing_object.file_ids:
            updates = dict()
            step_index = self.determine_correct_step_index(database = processing_object.database, file_id = file_id)
            updates[f'{self.processing_type}_step_{str(step_index).zfill(2)}'] = self.strategy_name
            updates = self.add_strategy_specific_infos_to_updates(updates = updates)
            processing_object.database.update_file_infos(file_id = file_id, updates = updates)
        return processing_object


    @property
    def strategy_name(self):
        return self.__class__.__name__ 
            
    
    def determine_correct_step_index(self, database: Database, file_id: str) -> int:
        file_infos = database.get_file_infos(identifier = file_id)
        previous_step_indices_of_same_processing_type = []
        for key, value in file_infos.items():
            if f'{self.processing_type}_step_' in key:
                if value != None: # to ensure that this file_id was actually already processed
                    step_index = int(key[key.rfind('_') + 1 :])
                    previous_step_indices_of_same_processing_type.append(step_index)
        if len(previous_step_indices_of_same_processing_type) > 0:
            correct_step_index = max(previous_step_indices_of_same_processing_type) + 1
        else:
            correct_step_index = 0
        return correct_step_index

Furthermore, the following two classes `DataLoader` and `DataReader` will be re-used throughout the *findmycells* package to load data into your *findmycells* project.

In [ ]:
#| export

class DataReader(ABC):
    
    """
    Abstract base class that defines the general structure of DataReader subclasses.
    Essentially, it demands the corresponding subclasses to define the "readable_filetype_extensions" 
    attribut, as well as the "set_optional_configs()" and the "read()" methods.
    """
    
    @property
    @abstractmethod
    def readable_filetype_extensions(self) -> List[str]:
        """
        Property that will denote which filetype extensions the respective DataReader subclass can handle.
        """
        pass
    
    
    @property
    @abstractmethod
    def key_to_configs_attribute_in_database(self) -> str:
        """
        Property that will specify the key under which the corresponding configs can be found in the project configs,
        for instance "MicroscopyImageReader_configs". Will be specified in the DataReader subclass in each DataReader
        module, from which all correpsonding DataReaders of this subclass will inherit from.
        """
        pass
    
    
    @property
    @abstractmethod
    def default_config_values(self) -> Dict[str, Any]:
        """
        Property that will specify all configs that apply to the specific DataReader subclass.
        """
        pass


    @property
    @abstractmethod
    def default_config_value_types(self) -> Dict[str, List]:
        """
        Property that will specify the types of each configs value.
        """
        pass
    
    
    @abstractmethod
    def read(self, filepath: Path, database: Database) -> Any:
        """
        This method eventually reads the data stored at the given filepath applying the specified configs.
        The returned datatype will be different for each DataReader subclass, e.g. a numpy array of a specific
        shape for MicroscopyImageReaders, or a shapely Polygon for ROIReaders.
        """
        pass
    
    
    @abstractmethod
    def assert_correct_output_format(self, output: Any) -> None:
        """
        Run an assert to validate that the data was actually read in the correct way and that the created output
        matches the intended format!
        """
        pass
    

    def set_optional_configs(self, database: Database) -> None:
        """
        For some data it will be important for the user to be able to specify additional configurations, like 
        for instance which color channel should be read, loaded, and eventually analyzed for microscopy images.
        These settings will be saved in the 'optional_configs' section of the `Database` of the project under
        the corresponding key, which is, again, specific for each of the data types and will be defined in each
        DataReader subclass (e.g. 'MicroscopyImageReader_configs' for all DataReaders that handle microscopy
        image data). Via the '_set_defaults_for_unspecified_or_invalid_configs()' method, missing values will
        be filled with default values (defined in 'self.default_config_values') and the types of all user
        specified values will be confirmed (as defined in 'self.default_config_value_types').
        """
        configs = database['optional_configs'][self.key_to_configs_attribute_in_database]
        if type(configs) == dict:
            if len(configs.keys) > 0:
                for key, value in configs.items():
                    setattr(self, key, value)
        self._set_defaults_for_unspecified_or_invalid_configs()
    

    def _set_defaults_for_unspecified_or_invalid_configs(self) -> None:
        for key, default_value in self.default_config_values.items():
            if hasattr(self, key):
                specified_value = getattr(self, key)
                if type(specified_value) not in self.default_config_value_types[key]:
                    raise ValueError(f'The type of the value you specified as {key} in the {self.key_to_configs_attribute_in_database} '
                                     f'does not match the expected type. Your specified value was: {specified_value}, which is of type: '
                                     f'{type(specified_value)}, while the default value is: {default_value}, which is of type: '
                                     f'{type(default_value)}.')
            else:
                setattr(self, key, default_value)

In [ ]:
#| export

class DataLoader:
    
    def determine_reader(self, file_extension: str, data_reader_module: ModuleType) -> DataReader:
        available_reader = None
        for name, data_reader in inspect.getmembers(data_reader_module):
            if name.endswith('Reader'):
                if file_extension in data_reader().readable_filetype_extensions:
                    available_reader = obj
        if available_reader == None:
            raise NotImplementedError(f'Unfortunately, there is no DataReader implemented in {data_reader_module} '
                                      f'which can handle your filetype ("{file_extension}").')
        return available_reader
    
    
    def load(self, data_reader_class: DataReader, filepath: Path, database: Database) -> Any:
        data_reader = data_reader_class()
        data_reader.set_optional_configs(database = database)
        data = data_reader.read(filepath = filepath, database = databse)
        data_reader.assert_correct_output_format(output = data)
        return data                 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()